In [1]:
import pandas as pd
import numpy as np
import re
import opencc
import jieba
import jieba.posseg as pseg

In [2]:
def clean(text):
    cop = re.compile("[^\u4e00-\u9fa5^a-z^A-Z^0-9]")  # 匹配不是中文、大小写、数字的其他字符
    t = cop.sub('', text)  # 将string1中匹配到的字符替换成空字符
    cc = opencc.OpenCC('t2s') # 繁体转简体
    return cc.convert(t)

In [3]:
df = pd.read_excel('./data/weibo12.xlsx')

In [4]:
df

,time,text
0,12月01日,\n 哈马斯发布来自也门的声援 ​​​ ​ ...
1,12月01日,\n 用无人机炸了 的船 ​ ...
2,12月01日,\n 在以色列军队恢复军事行动的1小时内，真主党武装在...
3,12月01日,\n 以色列再次空袭加沙以色列对加沙南部拉法的阿德万家...
4,12月01日,\n 卡桑旅的火箭弹在特拉维夫上空，铁穹紧急拦截 ​...
...,...,...
287,12月01日,\n 美国国务卿布林肯：以色列拥有可以攻击哈马斯而不...
288,12月01日,\n 不在了吗1: 巴以冲突升级，38小时超1000杦...
289,12月01日,\n 小鱼儿网服务众包平台为您收集的每日新闻（12.0...
290,12月01日,\n 以色列轰炸哈马德镇和汗尤尼斯的卡拉拉住宅楼后，媒...


In [5]:
df = df.drop_duplicates(keep='first')
df

,time,text
0,12月01日,\n 哈马斯发布来自也门的声援 ​​​ ​ ...
1,12月01日,\n 用无人机炸了 的船 ​ ...
2,12月01日,\n 在以色列军队恢复军事行动的1小时内，真主党武装在...
3,12月01日,\n 以色列再次空袭加沙以色列对加沙南部拉法的阿德万家...
4,12月01日,\n 卡桑旅的火箭弹在特拉维夫上空，铁穹紧急拦截 ​...
...,...,...
287,12月01日,\n 美国国务卿布林肯：以色列拥有可以攻击哈马斯而不...
288,12月01日,\n 不在了吗1: 巴以冲突升级，38小时超1000杦...
289,12月01日,\n 小鱼儿网服务众包平台为您收集的每日新闻（12.0...
290,12月01日,\n 以色列轰炸哈马德镇和汗尤尼斯的卡拉拉住宅楼后，媒...


In [6]:
df['text'] = df['text'].map(clean)
df = df.replace('', np.nan)
df = df.dropna(how='any', subset=['text']).reset_index(drop=True)
df

C:\Users\31426\AppData\Local\Temp\ipykernel_16936\388437319.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].map(clean)


,time,text
0,12月01日,哈马斯发布来自也门的声援
1,12月01日,用无人机炸了的船
2,12月01日,在以色列军队恢复军事行动的1小时内真主党武装在以色列北部边境确认发起了3次袭击抵抗之弧在临时...
3,12月01日,以色列再次空袭加沙以色列对加沙南部拉法的阿德万家进行空袭造成无辜巴勒斯坦平民伤亡
4,12月01日,卡桑旅的火箭弹在特拉维夫上空铁穹紧急拦截
...,...,...
246,12月01日,美国国务卿布林肯以色列拥有可以攻击哈马斯而不针对妇女和儿童的装备PS两个意思一加沙全员哈马斯...
247,12月01日,不在了吗1巴以冲突升级38小时超1000杦火箭弹支持巴勒斯坦人民的抗以战争
248,12月01日,小鱼儿网服务众包平台为您收集的每日新闻1201我国艾滋病感染率和死亡率均处全球较低水平以色列...
249,12月01日,以色列轰炸哈马德镇和汗尤尼斯的卡拉拉住宅楼后媒体摄像机拍摄下来的现场画面


In [7]:
df.to_excel('./data/weibo12月1日清洗过后.xlsx', index=False)

In [8]:
# 创建停用词列表
def stopwordslist():
    stopwords = [line.strip() for line in open('./data/stop_words_ch.txt', encoding='UTF-8').readlines()]
    return stopwords

In [9]:
# 对句子进行中文分词
def seg_depart(sentence):
    # 对文档中的每一行进行中文分词
    sentence_depart = pseg.cut(sentence.strip())
    # 创建一个停用词列表
    stopwords = stopwordslist()
    # 输出结果为outstr
    outstr = ''
    # 去停用词
    for word, flag in sentence_depart:
        if word not in stopwords:
            if flag[0] == 'n':
                # print(word, flag)
                outstr += word
                outstr += " "
    return outstr

In [10]:
df = pd.read_excel('./data/weibo12月1日清洗过后.xlsx')
jieba.load_userdict('./data/my_dic.txt')
df['text'] = df['text'].map(seg_depart)
df = df.replace('', np.nan)
df = df.dropna(how='any')
df.to_csv('./data/spilt.csv', index=False)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\31426\AppData\Local\Temp\jieba.cache
Loading model cost 0.590 seconds.
Prefix dict has been built successfully.


In [11]:
from datetime import datetime, timedelta

In [12]:
start_date = datetime(2023,10,7)
end_date = datetime.now()
end_date

datetime.datetime(2023, 12, 2, 15, 6, 59, 32368)

In [33]:
df1 = df[(df['time'] >= '10月07日') & (df['time'] <= '10月20日')]
df1.to_csv('./data/weibo_1.csv', index=False)
df1

,time,text
0,10月07日,人 哈马斯 鹅粉 利用 老弱妇孺 心情 巴勒斯坦 同情 哈马斯 事 人 解题
1,10月07日,巴勒斯坦 卫生部门 加沙 地带 巴勒斯坦人 人 人 现场 视频
2,10月07日,以色列 官方 以色列 人 人数 以色列 部门 哈马斯 以色列 人 官方 人数 人 人 人 官...
3,10月07日,以色列 客运 铁路 部队 装备 以色列 营 以色列 加沙 地带 边境 预备役 人员 武器 装...
4,10月07日,以色列 军方 哈马斯 以色列 人 加沙 地带 战争 罪行 外交官 联合国安理会 会议 展开讨...
...,...,...
3098,10月20日,得国 好下场
3099,10月20日,战争
3100,10月20日,平民百姓
3101,10月20日,哈马斯 旗下 卡萨姆 旅 坦克 杀器


In [34]:
df2 = df[(df['time'] >= '10月21日') & (df['time'] <= '11月03日')]
df2.to_csv('./data/weibo_2.csv', index=False)
df2

,time,text
3103,10月22日,加拿大 总理 特鲁多 多伦多 清真寺 电视台 穆斯林 多伦多 清真寺 加拿大 总理 贾 斯汀...
3104,10月21日,以色列 加沙 地带
3105,10月21日,以色列 地面 人质 国际 情势 费思量 老汤 话
3106,10月21日,德国 城市 示威游行 声援 加沙 哈马斯
3107,10月21日,真主党 火箭 导弹
...,...,...
6381,11月03日,赵 俊杰 欧洲 现实 因素 欧洲 难民 欧洲 社会 乌克兰 难民 沉重负担 欧洲 巴勒斯坦 ...
6382,11月03日,巴
6383,11月03日,以色列国防军 加沙 布拉吉 难民营 废墟 孩子 父亲 工具 徒手 瓦砾 家庭
6384,11月03日,孕妇 加沙 医院 剖腹产 麻药 新生儿 死亡威胁 无辜者


In [35]:
df3 = df[(df['time'] >= '11月04日') & (df['time'] <= '11月17日')]
df3.to_csv('./data/weibo_3.csv', index=False)
df3

,time,text
6386,11月05日,玻利维亚 加沙 人道主义 物资 玻利维亚政府 加沙 地带 人道主义 物资 玻利维亚 意见 报...
6387,11月04日,魔爪 心脏
6388,11月04日,陈慧慧 总台 土耳其 安卡拉 记者 陈慧慧 现场 陈慧慧 父亲 陈恩富 记者 女儿 前线 战...
6389,11月04日,头 网络 媒体 中毒 美 物资 学学
6390,11月04日,无辜 巴勒斯坦 人民 孩子 国家 以色列 极端 恐怖份子
...,...,...
9912,11月17日,犹太 自导自演 世界 人民 仗 爹 横行霸道
9913,11月17日,莱德 乌克兰 国有资产 美国 巴以 美国 美国 理由 欧洲 盟友 交代 利益 利益 犹太人 利益
9914,11月17日,以色列 士兵 哈马斯 枪战 视频 哈马斯 武装 分子 身影 以色列 士兵 枪 对准 地道口 ...
9915,11月17日,明白 历史 厚重感 感觉 民族 信仰 心酸 血泪 历史 局面 历史 成 仇恨 隔阂 阴谋论 ...


In [36]:
df4 = df[(df['time'] >= '11月18日') & (df['time'] <= '11月30日')]
df4.to_csv('./data/weibo_4.csv', index=False)
df4

,time,text
9917,11月18日,伊玛目 时 以色列 士兵 清真寺 手榴弹 事件 约旦河西岸 拉马拉 巴勒斯坦 村庄 布 德鲁...
9918,11月18日,图书馆 耳机 视频 巴勒斯坦 队伍 散场 一家人 爸爸 国旗 小朋友 旗子 样子 棉袄 头发...
9919,11月18日,加沙 民众 军队 哈马斯 地方 加沙 哈马斯 加沙 地带 加沙 地带 人道主义 危机 卫星 ...
9920,11月18日,贵大 烟花 力度 区别 原因 马斯克 中站 人道主义 做人 人 人
9921,11月18日,半岛 电视台 以色列 加沙 学校 人
...,...,...
13170,11月30日,规模 烈度 巴勒斯坦 浏览量 作者 史者
13171,11月30日,建议
13172,11月30日,高级别 会议 国际 社会 心声
13173,11月30日,法国 顾问 法国 媒体 文章


In [13]:
df1 = df[(df['time'] >= '12月01日') & (df['time'] <= '12月01日')]
df1.to_csv('./data/weibo_12.csv', index=False)
df1

,time,text
0,12月01日,哈马斯 声援
1,12月01日,船
2,12月01日,以色列 军队 军事行动 小时 内 真主党 武装 以色列 边境 弧 高度 戒备状态 包皮 佬 ...
3,12月01日,以色列 加沙 以色列 加沙 拉法 阿德 无辜 巴勒斯坦 平民
4,12月01日,卡桑 旅 火箭弹 特拉维夫 铁穹
...,...,...
246,12月01日,美国 国务卿 布 林肯 以色列 哈马斯 妇女 儿童 装备 加沙 全员 哈马斯 以色列 妇女儿...
247,12月01日,小时 火箭弹 巴勒斯坦 人民 战争
248,12月01日,小鱼儿 网 众 平台 新闻 艾滋病 感染率 死亡率 全球 低水平 以色列 加沙 地带 军事 ...
249,12月01日,以色列 哈马德 镇和汗 尼斯 卡拉拉 住宅楼 媒体 摄像机 现场 画面


In [28]:
df1 = pd.read_csv('./data/weibo_4_old.csv')
df2 = pd.read_csv('./data/weibo_12.csv')
df = pd.concat([df1, df2], ignore_index=True)
df.to_csv('./data/weibo_4.csv', index=False)